In [ ]:
library(foreach)
library(Matrix)
library(ggrepel)
library(ggplot2)
library(ggpubr)
library(cowplot)
library(doParallel)
library(plyr)
library(MASS)
library(NMF)
library(circlize)
library(plotly)
setwd('/home/projects/amit/annaku/repos/Blueprint/scripts')
theme_set(theme_cowplot())

In [ ]:
registerDoParallel(cores = 32)
getDoParWorkers()

In [ ]:
zscore_path <- '/home/projects/amit/annaku/repos/Blueprint/data/processed/zscore_outputs'
nmf_save_path <- '/home/projects/amit/annaku/repos/Blueprint/data/processed/nmf_outputs'

# load

In [ ]:
ver <- '20250306'
dat <- read.table(paste0(zscore_path, '/zstat_Atlas_v_', ver, '_full_samplelevel', '_preproc.txt'),
sep = '\t', header = T, row.names = 1, check.names = F)
dim(dat)

In [ ]:
nruns <-  100 
min_k <- 4 
max_k <- 15 

path_tmp <- '/home/projects/amit/annaku/repos/Blueprint/data/processed/nmf_outputs/'

file_path <- paste0(nmf_save_path, "/sim_ver_", ver, '_', nruns, "_runs_", min_k, "_", max_k, "_withgenes.csv")
sim_tosave <- read.csv(file_path)

# genes additional filtering

In [ ]:
resolution_list <- c(6)
resolution_string <- paste(resolution_list, collapse = "_")
thr_stability <- 0.55

In [ ]:
sim_tosave_filtered <- sim_tosave %>% filter(k %in% resolution_list)

genes_scores_dict <- list()

for (gene in rownames(dat)) {
  mask <- grepl(paste0(gene, ","), sim_tosave_filtered$intersected_genes)
  score <- sum(mask) / nrow(sim_tosave_filtered)
  genes_scores_dict[[gene]] <- score
}

scores_vector <- unlist(genes_scores_dict)

ggplot(data.frame(scores = scores_vector), aes(x = scores)) +
  geom_histogram(bins = 50, fill = "skyblue", color = "black") +
  theme_minimal() +
  labs(x = "Scores", y = "Count", title = "Histogram of Gene Scores")

most_stable_genes <- names(scores_vector[scores_vector > thr_stability])
print(length(most_stable_genes))

In [ ]:
scores_vector['NSD2']

In [ ]:
ggboxplot(sim_tosave, x = "k", y = "gene", ylim = c(0,1),
   add = "jitter"
   )

In [ ]:
ggboxplot(sim_tosave, x = "k", y = "cl", ylim = c(0,1), add = "jitter")

# multiple nmf running, recheck k

In [ ]:
dat_filtered = dat
dim(dat_filtered)

In [ ]:
set.seed(10)
multires <- nmf(dat_filtered, rank=5:10, rng = 10, nrun = 100, 
.options='vp3'
)

In [ ]:
saveRDS(multires, file = file.path(nmf_save_path, sprintf("multires_thr_%.2f_res_%s_high_res_samples_%s_Jan_2026.rds", thr_stability, resolution_string, POST_FIX)))

In [ ]:
library("viridis")
magma_palette <- rev(rocket(100))

options(repr.plot.width=14, repr.plot.height=8)
if(requireNamespace("Biobase", quietly=TRUE)){
consensusmap(multires, #annCol=dat, 
color = magma_palette,
labCol=NA, labRow=NA)
}

In [ ]:
# # save

# resolution_string <- paste(resolution_list, collapse = "_")
# summary_filename <- file.path(nmf_save_path, sprintf("/nmf_summary_thr_%.2f_res_%s_v%s.txt", thr_stability, resolution_string, ver))
# summary_output <- capture.output(summary(multires))
# summary_df <- data.frame(Summary = summary_output)
# #summary_df

# write.table(summary_df, file = summary_filename, sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)

In [ ]:
plot(multires)

In [ ]:
# # save

# resolution_string <- paste(resolution_list, collapse = "_")
# summary_filename <- file.path(nmf_save_path, sprintf("nmf_multi_rank_measures_plot_thr_%.2f_res_%s_v%s.png", thr_stability, resolution_string, ver))
# png(file=summary_filename, width=700, height=700)
# plot(multires)
# dev.off()

# transposed

In [ ]:
multires_t <- nmf(t(dat_filtered), rank=5:10, rng = 10, nrun = 100, 
.options='vp30')

In [ ]:
# saveRDS(multires_t, file = file.path(nmf_save_path, sprintf("multires_transposed__thr_%.2f_res_%s_high_res_samples_v%s.rds", thr_stability, resolution_string, ver)))

In [ ]:
# or load previously calculated

multires_t <- readRDS(file.path(nmf_save_path, 
                               sprintf("multires_transposed__thr_%.2f_res_%s_high_res_samples_v%s.rds", 
                                     thr_stability, resolution_string, ver)))

In [ ]:
library("viridis")
magma_palette <- rev(rocket(100))

options(repr.plot.width=15, repr.plot.height=8)
if(requireNamespace("Biobase", quietly=TRUE)){
consensusmap(multires_t, 
color = magma_palette,
labCol=NA, labRow=NA)
}

In [ ]:
magma_palette <- colorRampPalette(c(
    "#FFFFFF",               
    "#FFE5E5",              
    rocket(100)[30:70],      
    "#1A1A1A"            
))(100)

In [ ]:
selected_rank <- 6
ranks <- sapply(multires_t$fit, nbasis)
rank_index <- which(ranks == selected_rank)
nmf_model <- multires_t$fit[[rank_index]]

consensus_matrix <- consensus(nmf_model)

library("viridis")
magma_palette <- rev(rocket(100)[20:100])

options(repr.plot.width=8, repr.plot.height=8)
if(requireNamespace("Biobase", quietly=TRUE)){
  consensusmap(consensus_matrix,
               color = magma_palette,
               labCol=NA, labRow=NA,
               #Rowv = TRUE, 
               #Colv = TRUE, 
               main = paste("Genes modules Consensus Matrix for Rank", selected_rank))
}

In [ ]:
genes_of_interest <- c("PPIB", "APOBEC3G", 'CTSH',
"CCND1", 'FCRLA', 'BCL2',
'CDR1', 'HLA-H', 'CCPG1',
'ITGB7', 'MAF', 'MAFB',
'COX5A', 'RACK1', 'EEF2',
'BTLA', 'NSD2', 'FGFR3',
'PPIA', 'COX6C', 'MIF',
'MKI67', 'TOP2A'
)  
coef_matrix <- coef(nmf_model)
gene_clusters <- apply(coef_matrix, 2, which.max)


gene_cluster_assignments <- data.frame(
    Gene = genes_of_interest,
    Cluster = gene_clusters[genes_of_interest]
)

print(gene_cluster_assignments)

In [ ]:
cluster_within_group <- function(genes, matrix) {
  if (length(genes) > 1) {

    dist_matrix <- dist(matrix[genes, ])
    hc <- hclust(dist_matrix)
    return(genes[hc$order])
  }
  return(genes)
}

gene_clusters <- apply(coef_matrix, 2, which.max)


desired_cluster_order <- c(6, 3, 5, 2, 7, 1, 4)
ordered_genes <- unlist(lapply(desired_cluster_order, function(cluster) {
  genes_in_cluster <- names(gene_clusters[gene_clusters == cluster])
  cluster_within_group(genes_in_cluster, consensus_matrix)
}))


ordered_consensus_matrix <- consensus_matrix[ordered_genes, ordered_genes]


options(repr.plot.width=12, repr.plot.height=12)
if(requireNamespace("Biobase", quietly=TRUE)){
  consensusmap(ordered_consensus_matrix,
               color = magma_palette,
               labCol = ordered_genes,  
               labRow = ordered_genes,
               fontsize = 8,
               #Rowv = NA, 
               #Colv = NA,
               main = paste("Ordered Genes Modules Consensus Matrix"))
}

In [ ]:
options(repr.plot.width=2, repr.plot.height=3)

par(mar=c(1, 1, 1, 4))

plot(0, 0, type="n", xlim=c(0,1), ylim=c(0,1), 
     xlab="", ylab="", xaxt="n", yaxt="n", 
     bty="n")

n <- length(magma_palette)
rect(0, (0:(n-1))/n, 1, (1:n)/n, col=magma_palette, border=NA)

axis(4, las=1)

# single nmf run for chosen k

In [ ]:
dat_filtered = dat[most_stable_genes, ]
#dat_filtered = dat
dim(dat_filtered)

In [ ]:
library(synchronicity)

In [ ]:
k <- 6
set.seed(10)
res_cl <- nmf(dat_filtered, rank=k, rng = 10, nrun = 100, .options='v3p')
basis_matrix <- basis(res_cl)
coef_matrix <- coef(res_cl)

rownames(coef_matrix) <- 1:k
colnames(basis_matrix) <- 1:k

In [ ]:
sample_clusters <- apply(coef_matrix, 2, which.max)
cluster_assignments <- data.frame(sample = colnames(dat), clust = sample_clusters)

gene_clusters <- apply(basis_matrix, 1, which.max)
ordered_genes <- order(gene_clusters)

In [ ]:
library(pheatmap)

In [ ]:
cluster_assignments_sorted <- cluster_assignments[order(cluster_assignments$clust), ]
coef_matrix_sorted <- coef_matrix[, rownames(cluster_assignments_sorted)]

pheatmap(coef_matrix_sorted,
         #annotation_col = cluster_assignments_sorted,
         show_colnames = FALSE,
         cluster_cols = FALSE,
         main = "Sample Contributions to Clusters",
         annotation_legend = FALSE)

In [ ]:
library("viridis")
magma_palette <- rev(magma(100))

In [ ]:
sample_dataframe = merge(t(coef_matrix), annotation_col, by = "row.names")
sample_dataframe$index = sample_dataframe$Row.names

genes_dataframe = merge(basis_matrix, annotation_row, by = "row.names")
genes_dataframe$index = genes_dataframe$Row.names

In [ ]:
resolution_string <- paste(resolution_list, collapse = "_")
sample_filename <- file.path(nmf_save_path, sprintf("arch_sample_ver_%s_thr_%.2f_res_%s.csv", ver, thr_stability, resolution_string))
#write.csv(sample_dataframe, sample_filename, row.names = TRUE)


genes_filename <- file.path(nmf_save_path, sprintf("arch_gene_ver_%s_thr_%.2f_res_%s.csv", ver, thr_stability, resolution_string))
#write.csv(genes_dataframe, genes_filename, row.names = TRUE)

print(paste("Sample contributions saved to:", sample_filename))
print(paste("Gene contributions saved to:", genes_filename))